# 2.5 Mixed formulation for second order equations

Motivation:
* exact flux conservation
* useful for a posteriori error estimates
* model problem for $4^{th}$ order problems, Stokes, ...


We consider the diffusion equation
$$
\DeclareMathOperator{\Div}{div}
\begin{array}{rcll}
-\Div \lambda \nabla u & = & f & \text{ in } \Omega \\
u & = & u_D & \text{ on } \Gamma_D \\
\lambda \frac{\partial u}{\partial n} & = & g & \text{ on } \Gamma_N
\end{array}
$$

### Primal variational formulation

Find $u \in H^1,  u = u_D$ on $\Gamma_D$ such that

$$
\int_\Omega \lambda \nabla u \nabla v = \int_\Omega f v + \int_{\Gamma_N} g v
\quad \forall v \in H^1, v = 0 \text{ on } \Gamma_D
$$

### First order system

Find scalar $u$ and the flux $\sigma$ such that

$$
\lambda^{-1} \sigma = \nabla u, \quad \Div \sigma = -f
$$

with boundary conditions

$$
\sigma \cdot n = g \text{ on } \Gamma_N, \quad \text{ and } \quad
u = u_D \text{ on } \Gamma_D 
$$


### Mixed variational formulation

Find $\sigma \in H(\Div)$ and $u \in L_2$ such that $\sigma_n = g$ on $\Gamma_N$ and

\begin{eqnarray*}
\int_\Omega \lambda^{-1} \sigma \tau + \int_\Omega \Div \tau u & = & 0 \\
\int_\Omega \Div \sigma v  & = &  -\int_\Omega f v + \int_{\Gamma_D} u_D \tau_n
\end{eqnarray*}

for all test-functions $\tau \in H(\Div)$ and $v \in L_2$ with $\tau_n = 0$.

Here $\sigma_n$ is the normal trace operator $\sigma \cdot n |_{\Gamma_N}$, which is meaningful in $H(\Div)$.


A Compact notation is the single-liner 

Find $(\sigma, u) \in H(\Div) \times L_2$ such that $\sigma_n = g$ on $\Gamma_N$ and

$$
\int_\Omega \lambda^{-1} \sigma \tau + \Div \sigma v + \Div \tau u = 
-\int_\Omega f v + \int_{\Gamma_D} u_D \tau_n
$$

for all test-functions $(\tau, v) \in H(\Div) \times L_2$ with $\tau_n = 0$.


In [1]:
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui
%gui tk

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2305
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 8 thread(s)
 Generate Mesh from spline geometry


In [2]:
source = sin(3.14*x)
ud = CF(5)
g = mesh.BoundaryCF( {"left" : y*(1-y)}, default=0)
lam = 10

In [24]:
help(mesh.BoundaryCF)

Help on method BoundaryCF in module ngsolve.comp:

BoundaryCF(...) method of ngsolve.comp.Mesh instance
    BoundaryCF(self: ngsolve.comp.Mesh, values: dict, default: ngsolve.fem.CoefficientFunction = None) -> ngsolve.fem.CoefficientFunction
    
    Boundary wise CoefficientFunction.
    First argument is a dict from either boundary names or Region objects to
    CoefficientFunction (-values). Later given names/regions override earlier
    values. Optional last argument (default) is the value for not given boundaries.
    >>> penalty = mesh.BoundaryCF({ "top" : 1e6 }, default=0)
    will create a CF being 1e6 on the top boundary and 0. elsewhere.



Setup and solve primal problem:

In [30]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
fesp = H1(mesh, order=5, dirichlet="bottom")
up, vp = fesp.TnT()

ap = BilinearForm(lam*grad(up)*grad(vp)*dx).Assemble()
fp = LinearForm(source*vp*dx + g*vp * ds).Assemble()

gfup = GridFunction(fesp)
gfup.Set(ud, BND)

r = fp.vec - ap.mat * gfup.vec
gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
Draw (gfup)
Draw (lam * grad(gfup), mesh, "flux-primal")
exact = gfup
grad_exact = grad(exact)

 Generate Mesh from spline geometry


AttributeError: 'ngsolve.fem.CoefficientFunction' object has no attribute 'vec'

In [34]:
mesh_size = [0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625]
order = 2
dofs = []
errors = []
grad_errors = []


for h in mesh_size:
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fesp = H1(mesh, order=1, dirichlet="bottom")
    up, vp = fesp.TnT()

    ap = BilinearForm(lam*grad(up)*grad(vp)*dx).Assemble()
    fp = LinearForm(source*vp*dx + g*vp * ds).Assemble()

    gfup = GridFunction(fesp)
    gfup.Set(ud, BND)

    r = fp.vec - ap.mat * gfup.vec
    gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * r
    # Draw (gfup)
    # Draw (lam * grad(gfup), mesh, "flux-primal")

    # Compute error
    grad_gfup = grad(gfup)
 
    # Compute error
    err = sqrt (Integrate ( (gfup-exact)*(gfup-exact), mesh))
    grad_err = sqrt (Integrate ( (grad_gfup-grad_exact)*(grad_gfup-grad_exact), mesh))
    print ("h = ", h, "L2-error:", err, "grad-error:", grad_err)

    errors.append(err)
    grad_errors.append(grad_err)



 Generate Mesh from spline geometry
h =  0.5 L2-error: 0.004789109639791698 grad-error: 0.01963820993956928
 Generate Mesh from spline geometry
h =  0.25 L2-error: 0.00047243639292102727 grad-error: 0.006488385652396256
 Generate Mesh from spline geometry
h =  0.125 L2-error: 0.00011849539145395955 grad-error: 0.0033372029501114804
 Generate Mesh from spline geometry
h =  0.0625 L2-error: 2.5192650282231653e-05 grad-error: 0.0015591336363925514
 Generate Mesh from spline geometry
h =  0.03125 L2-error: 6.2992634229284545e-06 grad-error: 0.0007812040591427369
 Generate Mesh from spline geometry
h =  0.015625 L2-error: 1.5600753011396723e-06 grad-error: 0.00038892865429252963


In [32]:
import numpy as np

In [35]:
for i in range(len(mesh_size)-1):
    conv_rate = np.log(errors[i+1]/errors[i])/np.log(mesh_size[i+1]/mesh_size[i])
    print("Convergence rate:", conv_rate)
    print("Convergence rate for grad errors:", np.log(grad_errors[i+1]/grad_errors[i])/np.log(mesh_size[i+1]/mesh_size[i]))

Convergence rate: 3.3415654559583383
Convergence rate for grad errors: 1.5977319540561878
Convergence rate: 1.9952891526776657
Convergence rate for grad errors: 0.959220146190252
Convergence rate: 2.23375614306561
Convergence rate for grad errors: 1.0978948363235137
Convergence rate: 1.999747854163129
Convergence rate for grad errors: 0.9969732389361131
Convergence rate: 2.0135674769410072
Convergence rate for grad errors: 1.0061939163513178


## Solving the mixed problem
Define spaces for mixed problem. Note that essential boundary conditions  are set to the $H(\Div)$-component on the opposite boundary. Creating a space from a list of spaces generates a product space:

In [14]:
order_flux=1
V = HDiv(mesh, order=order_flux, dirichlet="right|top|left")
Q = L2(mesh, order=order_flux-1)
fesm = V*Q

The space provides now a list of trial and test-functions:

In [15]:
sigma, u = fesm.TrialFunction()
tau, v = fesm.TestFunction()

The normal vector is provided as a *special* coefficient function (which may be used only at the boundary). The orientation depends on the definition of the geometry. In 2D, it is the tangential vector rotated to the right, and is the outer vector in our case. Since every CoefficientFunction must know its vector-dimension, we have to provide the spatial dimension:

In [16]:
normal = specialcf.normal(mesh.dim)
print (normal)

coef normal vector, real, dim=2



Define the forms on the product space. For the boundary term, we have to use the Trace operator, which provides the projection to normal direction. 

In [17]:
am = BilinearForm((1/lam*sigma*tau + div(sigma)*v + div(tau)*u)*dx).Assemble()
fm = LinearForm(-source*v*dx + ud*(tau.Trace()*normal)*ds).Assemble()

gfm = GridFunction(fesm)

The proxy-functions used for the forms know to which component of the product space they belong to. To access components of the solution, we have to unpack its components. They don't have own coefficient vectors, but refer to ranges of the big coefficient vector.

In [18]:
gfsigma, gfu = gfm.components

Just to try something:

In [19]:
gfsigma.Set(CF( (sin(10*x), sin(10*y)) ))
gfu.Set (x)
Draw (gfsigma, mesh, "flux-mixed")
Draw (gfu, mesh, "u-mixed");

Now set the essential boundary conditions for the flux part:

In [20]:
gfsigma.Set(g*normal, BND)

In [21]:
res = fm.vec.data - am.mat * gfm.vec
gfm.vec.data += am.mat.Inverse(freedofs=fesm.FreeDofs(), inverse="umfpack") * res
# solvers.BVP(bf=am, lf=fm, gf=gfm)
Draw (gfsigma, mesh, "flux-mixed")
Draw (gfu, mesh, "u-mixed");

Calculate the difference:

In [22]:
print ("err-u:   ", sqrt(Integrate( (gfup-gfu)**2, mesh)))
errflux = lam * grad(gfup) - gfsigma
print ("err-flux:", sqrt(Integrate(errflux*errflux, mesh)))

err-u:    0.0010223748524829696
err-flux: 0.025306530356341964


## Post-processing for the scalar variable

The scalar variable is approximated one order lower than the vector variable, what is its gradient. Knowing the gradient of a function more accurate, and knowing its mean value, one can recover the function itself. For this post-processing trick we refer to [Arnold+Brezzi 85]


find $\widehat u \in P^{k+1, dc}$ and $\widehat \lambda \in P^{0, dc}$ such that

$$
\begin{array}{ccccl}
\int \lambda \nabla \widehat u \nabla \widehat v & + & \int \widehat \lambda \widehat v & = & \int \sigma \nabla \widehat v & \forall \, \widehat v\\
\int \widehat u \widehat \mu & & & = & \int u \widehat \mu & \forall \, \widehat \mu
\end{array}
$$

In [23]:
fespost_u = L2(mesh, order=order_flux+1)
fespost_lam = L2(mesh, order=0)
fes_post = fespost_u*fespost_lam

(u,la),(v,mu) = fes_post.TnT()

a = BilinearForm( (lam*grad(u)*grad(v)+la*v+mu*u)*dx).Assemble()
f = LinearForm((gfsigma*grad(v)+gfu*mu)*dx).Assemble()

gfpost = GridFunction(fes_post)
gfpost.vec.data = a.mat.Inverse() * f.vec

Draw (gfpost.components[0], mesh, "upost")
print ("err-upost:   ", sqrt(Integrate( (gfup-gfpost.components[0])**2, mesh)))

err-upost:    6.650960109439488e-05
